<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Evaluating the Privacy of a Function
Write a  function that calculates the **"sensitivity"of the mean() function** we use in Project 2 (for the query when measuring the difference between each parallel db's query result and the query result for the entire database (max value)).

##GLOSSARY:

**L1 Sensitivity** (or in short sensitivity) is the maximum amount that the query changes when removing an individual from the database. 

In [0]:
# importing torch
import torch

In [0]:
def get_paraller_db(db, remove_index):
  

  return torch.cat((db[0:remove_index], 
                    db[remove_index+1:]))

In [58]:
# the number of entries in database
num_entries = 5000

# will cretae a random tensor with 0's and 1's 
db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 0,  ..., 0, 1, 1], dtype=torch.uint8)

In [0]:
remove_index = 2

In [59]:
get_paraller_db(db,50).shape

torch.Size([4999])

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()

  for i in range(len(db)):
     pdb = get_paraller_db(db, i)
     parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
pdbs = get_parallel_dbs(db)

In [0]:
def create_db_and_parallels(um_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [0]:
def sensitivity(query, n_entires=1000):

  db, pdbs = create_db_and_parallels(20)
  
  full_db_result = query(db)
  
  max_distance = 0
  for pdb in pdbs:
    pdb_result = query(pdb)
  
    db_distance = torch.abs(pdb_result - full_db_result)
  
    if(db_distance > max_distance):
      max_distance = db_distance
  return max_distance

In [0]:
def query (db):
   return db.sum()
   #return db.float().mean() # mean = sum/number of entries

In [24]:
sensitivity(query)

tensor(1)

# Task: 
- Create the quesry() function
- Create 10 dtaabases of size 10
- Query each database with a threashols of 5 (calculate sensitivity)
- Print out the sensitivity of each database

In [0]:
def query_treshold(db, treshold=5):
  return (db.sum() > treshold).float()

In [55]:
for i in range(10):
  sens_f = sensitivity(query_treshold, n_entires=10)
  print(sens_f)

0
0
0
0
0
0
0
0
0
0


# Task: Performan a Differencing Attack on Row 10
Construct a database and then demonstrate how how by using two different queries expose the value of teh person represented by row 10 in teh database (note, a database with at least 10 rows is needed)

In [0]:
# Initilization of db with 10 entries
db, pdbs = create_db_and_parallels(30)

In [74]:
sum(db)

tensor(178, dtype=torch.uint8)

In [70]:
pdbs[10]

tensor([0, 0, 1,  ..., 0, 1, 0], dtype=torch.uint8)

In [71]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(224, dtype=torch.uint8)

In [73]:
# differencing attack using mean query
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0064)

In [75]:
# differencig atack using treshold
(sum(db).float() > 177) - (sum(pdb).float() > 177)

tensor(0, dtype=torch.uint8)